In [ ]:
##第二章：简单的应用以及神经网络的介绍

In [ ]:
#第一个案例，MNIST入门

In [24]:
import tensorflow as tf
import input_data #自己定义的文件
print(tf.__version__)#1.10.0

1.10.0


In [25]:
#下载&读取MNIST的数据，需要使用VPN
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)#下载四个数据包

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [32]:
#训练模型
x=tf.placeholder("float",[None,784])#创建一个占位符x，形状是[None，784],None代表可以是任意长度的
W=tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y=tf.nn.softmax(tf.matmul(x,W)+b)
y_=tf.placeholder("float",[None,10])
cross_entropy=-tf.reduce_sum(y_*tf.log(y))
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
init=tf.initialize_all_variables()
sess=tf.Session()
sess.run(init)
for i in range(1000):
    batch_xs,batch_ys=mnist.train.next_batch(100)
    sess.run(train_step,feed_dict={x:batch_xs,y_:batch_ys})
sess.close()

In [33]:
#评估模型
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))#检测预测的值是否和真实的值匹配
accuracy=tf.reduce_mean(tf.cast(correct_prediction,"float"))#比较之后取个平均值来表示相似度
print (sess.run(accuracy,feed_dict={x:mnist.test.images,y_:mnist.test.labels}))

RuntimeError: Attempted to use a closed Session.

In [34]:
#第二个案例 MNIST中介应用

In [37]:
import tensorflow as tf

In [44]:
sess=tf.InteractiveSession()#启动交互式的工作环境
x=tf.placeholder("float",shape=[None,784])#bach的大小即数据集的大小
y_=tf.placeholder("float",shape=[None,10])#每一张图片对应的图片类别
W=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))

In [45]:

sess.run(tf.initialize_all_variables())
#实现回归模型
#把向量化后的图片x和权重矩阵W相乘，再加上偏置b，计算每个分类色softmax的概率值
y=tf.nn.softmax(tf.matmul(x,W)+b)
#交叉熵
cross_entropy=-tf.reduce_sum(y_*tf.log(y))#计算的是整个minibatch的交叉熵


In [47]:
#训练模型
#最速下降法降低交叉熵
#更多的优化算法：http://www.tensorfly.cn/tfdoc/api_docs/python/train.html#optimizers
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
#反复运行train_step降低交叉熵
for i in range(1000):
    batch=mnist.train.next_batch(50)
    train_step.run(feed_dict={x:batch[0],y_:batch[1]})



In [50]:
#模型评估
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,"float"))
print(accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels}))

0.9116


In [62]:
#模型提升 
#权重初始化
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)
#卷积和池化
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#第一层卷积
#在每个5x5的patch中计算出32个特征，形状是[5,5,1,32]，前两个是pathc的大小，1是输入通道的数目，32是输出通道的数目
W_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])
#为了更好的利用第一层的卷积，我们将原来的一维图片数据拓展为4D向量，第二位第三位代表图怕你的宽高，最后一维代表颜色的通道数目，因为四灰度土图片，所以通道数维1
x_image=tf.reshape(x,[-1,28,28,1])
#使用relu激活，最后池化
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)
#第二层卷积
W_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)
#密集链接层
#缩小图片尺寸，加入1024个神经元进行全连接，处理整个图片
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])

h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

#DropOut
#为了减少过拟合现象，我们在输出层之前加入dropout
keep_prob=tf.placeholder("float")
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

#输出层
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)



In [66]:
#训练和评估
#计算交叉熵
cross_entropy=-tf.reduce_sum(y_*tf.log(y_conv))
#使用adam算法进行优化
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#预测
correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,"float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch=mnist.train.next_batch(50)
    if i%100==0:
        train_accuracy=accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
        print("step %d,training accuracy %g"%(i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})

print("test accuracy %d"%accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.t4st.labels,keep_prob:1.0}))

step 0,training accuracy 0.08
step 100,training accuracy 0.8


KeyboardInterrupt: 